# [Regev's LWE](https://cims.nyu.edu/~regev/papers/qcrypto.pdf) Public Key Cryptosystem

In [1]:
import numpy as np
from numpy.testing import assert_array_equal

In [2]:
# Modulus
q = 65536
# Lattice dimension
n = 3

## Secret Key

In [3]:
# Our LWE secret which is used below to construct the Public Key
sk = np.random.choice(q, n)
sk

array([ 1105, 19041, 47494])

In [4]:
# With `v` (a variant of our Secret Key `sk`) we can basically recover the error we add when creating our Public Key `pk`
v = np.append(-sk % q, [1])
v

array([64431, 46495, 18042,     1])

## Public Key

In [5]:
# This is the error we're introducing to make it hard to recover information from our Public Key `pk`
e = np.random.choice(q // 4, n)
e

array([ 5851, 12895,  7497])

In [6]:
# The matrix `A` is our System of Linear Equations
A = np.random.choice(q, (n, n))
A

array([[43390,   454, 35574],
       [35489, 32362, 33863],
       [27010, 23733, 10572]])

In [7]:
# Here we're computing results for our System of Linear equations (`A`) via our `sk` vector
# Note that we add the error term we've defined above
b = (A.dot(sk) + e) % q
b

array([ 6787, 43428, 33736])

In [8]:
# The public key is the "plain" matrix `A` in combination with an evaluation of such matrix via our `sk`
# We essentially append a "solution" column to our System of Linear Equations (which is our matrix `A`)
pk = np.column_stack((A, b))

# Here we test that we can use `v` (a modified version of our Secret Key `sk`) to recover the error `e`
# Recovering the error from the Public Key makes it possible to retain encrypted values
assert_array_equal(pk.dot(v) % q, e)

## Encryption

In [9]:
# Our message is a bit (0 or 1) rather than a number or a string of text
# `mu` is either `0` or `q // 2` which makes it possible to determine whether the bit was `0` or `1` once
# the error `e` was removed (when decrypting later on)
m = 1
mu = m * q // 2
mu

32768

In [10]:
# The `x` acts as a mask which determines which parts of the `pk` / `A` matrix we're about to evaluate
x = np.random.choice(2, n)
x

array([1, 0, 1])

In [11]:
# Using `emb` we're embedding the `mu` value into the "result column" when we're evaluating the `pk` / `A` matrix
emb = np.append(np.full(n, 0), mu)
emb

array([    0,     0,     0, 32768])

In [12]:
# Using our mask `x` we can evaluate our `pk` / `A` matrix and add the `emb` vector to it such that the `mu` value is
# embedded into the result which is our ciphertext
c = (x.dot(pk) + emb) % q
c

array([ 4864, 24187, 46146,  7755])

## Decryption

In [13]:
# Decryption is as simple as using our `sk` variant `v` to remove the noise from the ciphertext
p = c.dot(v) % q
p

46116

In [14]:
# We can now check if the value we're getting is closer to `q // 2` or `0`
# Closer to `q // 2` --> 1 was embedded
# Closer to `0`      --> 0 was embedded
if abs((p - (q // 2)) % q) < (p % q):
    print('The message is 1')
else:
    print('The message is 0')

print()
print(f'abs((p - (q // 2)) % q): {abs((p - q // 2) % q)}')
print(f'(p % q):\t\t {(p % q)}')

The message is 1

abs((p - (q // 2)) % q): 13348
(p % q):		 46116
